In [37]:
import re
import random
import string
import torch

In [436]:
import pandas as pd
train_data = pd.read_csv('/Users/lemarx/Documents/01_projects/SentencesRelatedness24/data/raw/eng_train.csv')
tada = train_data['Text'].iloc[200]
tada

'You try to forget everything good, and remember everything bad.\nyou just try to forget everything good, and remember everything bad.'

In [437]:
import nltk
from nltk.corpus import wordnet as wn
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/lemarx/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /Users/lemarx/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [440]:
# test = tada.split('\n')[0]
# seq = re.sub(r"[^a-zA-Z0-9]+", ' ', test)
# seq = list(set(seq.split()))

# candidates = [word for word in seq if len(word) >= 3]
# #words = nltk.word_tokenize(seq)
# pos_tags = nltk.pos_tag(candidates)


# word,tag = pos_tags[0]

# wn.synsets('good', pos=find_word_type(word_type_dict,'JJ'))


[Synset('good.a.01'),
 Synset('full.s.06'),
 Synset('good.a.03'),
 Synset('estimable.s.02'),
 Synset('beneficial.s.01'),
 Synset('good.s.06'),
 Synset('good.s.07'),
 Synset('adept.s.01'),
 Synset('good.s.09'),
 Synset('dear.s.02'),
 Synset('dependable.s.04'),
 Synset('good.s.12'),
 Synset('good.s.13'),
 Synset('effective.s.04'),
 Synset('good.s.15'),
 Synset('good.s.16'),
 Synset('good.s.17'),
 Synset('good.s.18'),
 Synset('good.s.19'),
 Synset('good.s.20'),
 Synset('good.s.21')]

In [586]:
#example to show the flaws in pos tagging
seq = nltk.word_tokenize("Time flies like an arrow; fruit flies like a banana.")
nltk.pos_tag(seq)

[('Time', 'NNP'),
 ('flies', 'NNS'),
 ('like', 'IN'),
 ('an', 'DT'),
 ('arrow', 'NN'),
 (';', ':'),
 ('fruit', 'CC'),
 ('flies', 'NNS'),
 ('like', 'IN'),
 ('a', 'DT'),
 ('banana', 'NN'),
 ('.', '.')]

In [535]:
#takes a pos_tagged sentence as function argument
def syn_replace_choice(pos_tags : list):
    replacable_tags = ['JJ', 'JJR', 'JJS','RB', 'RBR', 'RBS']
    filtered_data = [item for item in pos_tags if item[1] in replacable_tags]
    return filtered_data

In [607]:
def get_synonym(word : str, pos : str, seed : int = 42) -> str:
    synonyms = []
    if pos == None:
        return word
    for syn in wn.synsets(word, pos = pos):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())
    random.seed(seed)
    return random.choice(list(set(synonyms)))

In [608]:
def find_word_type(target_tag):
    word_type_dict = {wn.ADJ : ['JJ', 'JJR', 'JJS'],wn.ADV : ['RB', 'RBR', 'RBS']}
    for key, tag_list in word_type_dict.items():
        if target_tag in tag_list:
            return key
    return None  # Tag not found in any list

In [609]:
#takes the first sentence of the 2 sentences which are compared as an argument
#returns a tuple of of the random word and pos tag
#if used for syn replacement only certain words are allowed
def get_random_word(seq : str, min_len : int = 3,seed : int = None, sep = '[SEP]', syn_replace : bool = False) -> tuple:
    random.seed(seed)
    seq = seq.replace(sep, '')
    tokens = nltk.word_tokenize(seq)
    pos_tags = nltk.pos_tag(tokens)
    if syn_replace:
        pos_tags = syn_replace_choice(pos_tags)
    
    pos_tags = list(set(pos_tags))
    candidates = [word for word in pos_tags if len(word[0]) >= min_len]
    if(candidates == []):
        return None

    return random.choice(candidates)


In [610]:
def apply_syn_replacement(seq : str, sep : str = '[SEP]', seed : int = None, p : float = 0.3):
    word = get_random_word(seq, sep = sep, syn_replace= True, seed = seed)
    if word[0] != None:
        seq = seq.replace(word[0],get_synonym(word[0], pos = find_word_type(word[1]), seed = seed))
    return seq

In [612]:
#changes one random letter of the word
#dont interchange first or last letter
def replace_letter(word : str, seed : int = None)-> str:
    random.seed(seed)
    if len(word) <=2:
        return word
    idx = random.randint(1,len(word)-2)
    mod_word = word[:idx] + random.choice(string.ascii_lowercase) + word[idx + 1:] 
    return mod_word


In [613]:
def apply_change_letter(seq : str, p : int = 0.3, sep :str = '[SEP]', seed = None):
    seq = seq.split(sep)
    word = get_random_word(seq[0])[0]
    random.seed(seed)
    if word == None or random.random() < p:
        return seq[0] + sep + seq[1]
    return seq[0].replace(word,replace_letter(word,seed = seed)) + sep + seq[1]

In [687]:
print(apply_change_letter(tada, sep = '\n'))
print(apply_syn_replacement(tada, sep = '\n'))

You try to forget everything good, and remember everything byd.
you just try to forget everything good, and remember everything bad.
You try to forget everything good, and remember everything bad.
you simply try to forget everything good, and remember everything bad.


In [4]:
scores = torch.rand((1,32))
emb = torch.rand((32,265,786))

In [6]:
#function to select the 2 closes scores in the batch and return their indices within the batch
#def find_min_score_diff(emb : torch.Tensor,scores : torch.Tensor):
sorted, indices = torch.sort(scores, descending = True)
eps = 0.1
#if sorted[0][0] - sorted[0][1] < eps:
sorted   
#print(apply_interpol(torch.rand(1,265,786),sorted))

tensor([[0.9915, 0.9858, 0.8797, 0.8593, 0.8284, 0.8278, 0.8225, 0.8007, 0.7588,
         0.7584, 0.6318, 0.6290, 0.6231, 0.6166, 0.6037, 0.6014, 0.5955, 0.5313,
         0.4501, 0.4356, 0.4002, 0.3959, 0.3741, 0.3024, 0.3004, 0.1767, 0.1639,
         0.1283, 0.0794, 0.0556, 0.0404, 0.0093]])

In [331]:
emb[1].shape

torch.Size([265, 786])

In [51]:
def get_lowest_score_diff(scores : torch.Tensor)-> (int,int):
    differences = torch.abs(scores.view(-1, 1) - scores.view(1, -1))
    differences = differences.masked_fill(torch.eye(scores.numel()) == 1, float('inf'))
    min_indices = torch.argmin(differences)
    row_index = min_indices // differences.size(0)
    col_index = min_indices % differences.size(0)
    return row_index.item(),col_index.item()

In [69]:
#function for performing linear interpolation between 2 embeddings and there score
def apply_interpol(inputs : torch.Tensor, scores : torch.Tensor, alpha  : float = 0.5) -> (torch.Tensor,torch.Tensor):
    idx_a,idx_b = get_lowest_score_diff(scores)
    inputs[idx_b] = torch.lerp(inputs[idx_a], inputs[idx_b], alpha)
    scores[0][idx_b] = torch.lerp(scores[0][idx_a],scores[0][idx_b],alpha)
    return inputs,scores


In [68]:
scores[0][29].item

tensor(0.7588)

In [73]:
apply_interpol(inputs = emb, scores = scores, alpha=0.5)[1].shape


torch.Size([1, 32])